<a href="https://colab.research.google.com/github/Pigflyer55/flower_classifier_numerical/blob/main/keras_flower.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install -q sklearn

  Preparing metadata (setup.py) ... done


In [ ]:
%tensorflow_version 2.x

Colab only includes TensorFlow 2.x; %tensorflow_version has no effect.


In [ ]:
from __future__ import absolute_import, division, print_function, unicode_literals

import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from IPython.display import clear_output
from six.moves import urllib

import tensorflow.compat.v2.feature_column as fc

import tensorflow as tf

In [ ]:
CSV_COLUMN_NAMES = ['SepalLength', 'SepalWidth', 'PetalLength', 'PetalWidth', 'Species']
SPECIES = ['Setosa', 'Versicolor', 'Virginica']

In [ ]:
train_path = tf.keras.utils.get_file('iris_training.csv', "https://storage.googleapis.com/download.tensorflow.org/data/iris_training.csv")
test_path = tf.keras.utils.get_file('iris_test.csv', 'https://storage.googleapis.com/download.tensorflow.org/data/iris_test.csv')

train = pd.read_csv(train_path, names=CSV_COLUMN_NAMES, header=0)
test = pd.read_csv(test_path, names=CSV_COLUMN_NAMES, header=0)

573/573 [==============================] - 0s 0us/step


In [ ]:
train_label = train.pop('Species')
test_label = test.pop('Species')

In [ ]:
train.head(120)

,SepalLength,SepalWidth,PetalLength,PetalWidth
0,6.4,2.8,5.6,2.2
1,5.0,2.3,3.3,1.0
2,4.9,2.5,4.5,1.7
3,4.9,3.1,1.5,0.1
4,5.7,3.8,1.7,0.3
...,...,...,...,...
115,5.5,2.6,4.4,1.2
116,5.7,3.0,4.2,1.2
117,4.4,2.9,1.4,0.2
118,4.8,3.0,1.4,0.1


In [ ]:
train.describe().transpose()[['mean','std']]

,mean,std
SepalLength,5.845000,0.868578
SepalWidth,3.065000,0.427156
PetalLength,3.739167,1.822100
PetalWidth,1.196667,0.782039


In [ ]:
normalizer = tf.keras.layers.Normalization(input_shape=[4,], axis=None)

In [ ]:
normalizer.adapt(np.array(train))

In [ ]:
print(normalizer.mean.numpy())

[[3.4614584]]


In [ ]:
sepalLength = np.array(train['SepalLength'])
sepalLength_normalizer = tf.keras.layers.Normalization(input_shape=[1,], axis=None)
sepalLength_normalizer.adapt(sepalLength)

In [ ]:
sepalLength_model = tf.keras.Sequential([
    sepalLength_normalizer,
    tf.keras.layers.Dense(units=3, activation='softmax')
])
sepalLength_model.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 normalization_1 (Normalizat  (None, 1)                3         
 ion)                                                            
                                                                 
 dense (Dense)               (None, 3)                 6         
                                                                 
Total params: 9
Trainable params: 6
Non-trainable params: 3
_________________________________________________________________


In [ ]:
sepalLength_model.compile(optimizer=tf.keras.optimizers.Adam(learning_rate=0.1),
                          loss='sparse_categorical_crossentropy',
                          metrics=['accuracy'])


In [ ]:
history = sepalLength_model.fit(
    train['SepalLength'],
    train_label,
    epochs=100,
    validation_split=0.2
)

Epoch 1/100
3/3 [==============================] - 1s 103ms/step - loss: 1.3800 - accuracy: 0.2292 - val_loss: 1.3773 - val_accuracy: 0.2917
Epoch 2/100
3/3 [==============================] - 0s 13ms/step - loss: 1.1232 - accuracy: 0.4583 - val_loss: 1.0999 - val_accuracy: 0.1667
Epoch 3/100
3/3 [==============================] - 0s 16ms/step - loss: 0.9593 - accuracy: 0.5000 - val_loss: 0.8861 - val_accuracy: 0.6667
Epoch 4/100
3/3 [==============================] - 0s 13ms/step - loss: 0.8461 - accuracy: 0.6667 - val_loss: 0.7495 - val_accuracy: 0.6667
Epoch 5/100
3/3 [==============================] - 0s 14ms/step - loss: 0.7785 - accuracy: 0.6875 - val_loss: 0.6705 - val_accuracy: 0.6667
Epoch 6/100
3/3 [==============================] - 0s 13ms/step - loss: 0.7366 - accuracy: 0.6667 - val_loss: 0.6230 - val_accuracy: 0.6667
Epoch 7/100
3/3 [==============================] - 0s 15ms/step - loss: 0.7204 - accuracy: 0.6458 - val_loss: 0.5830 - val_accuracy: 0.7500
Epoch 8/100
3/3 [==

In [ ]:
hist = pd.DataFrame(history.history)
hist['epoch'] = history.epoch
hist.tail()

,loss,accuracy,val_loss,val_accuracy,epoch
95,0.658092,0.677083,0.453825,0.958333,95
96,0.656638,0.687500,0.451315,0.958333,96
97,0.657703,0.687500,0.450495,0.958333,97
98,0.661957,0.666667,0.454076,0.958333,98
99,0.657251,0.677083,0.449819,0.958333,99


In [ ]:
test_results = {}
test_results['sepalLength_model'] = sepalLength_model.evaluate(
    test['SepalLength'],
    test_label
)

1/1 [==============================] - 0s 52ms/step - loss: 0.6346 - accuracy: 0.7667


In [ ]:
flower_model = tf.keras.Sequential([
    normalizer,
    tf.keras.layers.Dense(units=64,activation='relu'),
    tf.keras.layers.Dense(units=3, activation='softmax')
])
flower_model.summary()

Model: "sequential_1"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 normalization (Normalizatio  (None, 4)                3         
 n)                                                              
                                                                 
 dense_1 (Dense)             (None, 64)                320       
                                                                 
 dense_2 (Dense)             (None, 3)                 195       
                                                                 
Total params: 518
Trainable params: 515
Non-trainable params: 3
_________________________________________________________________


In [ ]:
flower_model.compile(optimizer=tf.keras.optimizers.Adam(learning_rate=0.1),
                     loss='sparse_categorical_crossentropy',
                     metrics=['accuracy'])

In [ ]:
model_history = flower_model.fit(
    train,
    train_label,
    epochs=100,
    validation_split=0.2
)

Epoch 1/100
3/3 [==============================] - 1s 185ms/step - loss: 0.9427 - accuracy: 0.4688 - val_loss: 0.4769 - val_accuracy: 0.6667
Epoch 2/100
3/3 [==============================] - 0s 50ms/step - loss: 0.4178 - accuracy: 0.7708 - val_loss: 0.1526 - val_accuracy: 0.9583
Epoch 3/100
3/3 [==============================] - 0s 33ms/step - loss: 0.3263 - accuracy: 0.8125 - val_loss: 0.1425 - val_accuracy: 0.9583
Epoch 4/100
3/3 [==============================] - 0s 20ms/step - loss: 0.1912 - accuracy: 0.9062 - val_loss: 0.0826 - val_accuracy: 0.9583
Epoch 5/100
3/3 [==============================] - 0s 19ms/step - loss: 0.1845 - accuracy: 0.9062 - val_loss: 0.0584 - val_accuracy: 1.0000
Epoch 6/100
3/3 [==============================] - 0s 21ms/step - loss: 0.1212 - accuracy: 0.9583 - val_loss: 0.0662 - val_accuracy: 0.9583
Epoch 7/100
3/3 [==============================] - 0s 22ms/step - loss: 0.1111 - accuracy: 0.9583 - val_loss: 0.0471 - val_accuracy: 0.9583
Epoch 8/100
3/3 [==

In [ ]:
hist = pd.DataFrame(model_history.history)
hist['epoch'] = history.epoch
hist.tail()

,loss,accuracy,val_loss,val_accuracy,epoch
95,0.067026,0.958333,0.114813,0.958333,95
96,0.090432,0.968750,0.061517,0.958333,96
97,0.070899,0.968750,0.046666,1.000000,97
98,0.057909,0.989583,0.057369,0.958333,98
99,0.050730,0.979167,0.067506,0.958333,99


In [ ]:
v = 18

In [ ]:
print(test.iloc[v].to_numpy())
print(SPECIES[test_label[v]])

[5.1 3.4 1.5 0.2]
Setosa


In [ ]:
predicted = flower_model.predict(test.iloc[v].to_numpy())
print(predicted)
print(SPECIES[np.argmax(predicted)])
print(100*predicted[0][np.argmax(predicted)])
print(test_label)

1/1 [==============================] - 0s 19ms/step
[[9.9993277e-01 6.7232380e-05 8.2106998e-18]]
Setosa
99.99327659606934
0     1
1     2
2     0
3     1
4     1
5     1
6     0
7     2
8     1
9     2
10    2
11    0
12    2
13    1
14    1
15    0
16    1
17    0
18    0
19    2
20    0
21    1
22    2
23    1
24    1
25    1
26    0
27    1
28    2
29    1
Name: Species, dtype: int64


In [ ]:
test_loss, test_acc = flower_model.evaluate(test,  test_label, verbose=2)

print('\nTest accuracy:', test_acc)

1/1 - 0s - loss: 0.0437 - accuracy: 0.9667 - 18ms/epoch - 18ms/step

Test accuracy: 0.9666666388511658
